# Limpieza de datos

Limpieza de data set para Visualización de la información.

In [147]:
import pandas as pd
import geopandas as gpd
import re
import numpy as np
from warnings import WarningMessage

WarningMessage = False

In [111]:
# Dataset de Kaggle
data1 = pd.read_csv('data/15_to_30_victim_count.csv')
data2 = pd.read_csv('data/5_to_14_victim_count.csv')
data3 = pd.read_csv('data/Highest_victim_count.csv')
data4 = pd.read_csv('data/Lessthan_5_victim_count.csv')

#Mapa en geojson
mapa = gpd.read_file('https://raw.githubusercontent.com/simonepri/geo-maps/master/previews/countries-coastline.geo.json')

data = pd.concat([data1,data2,data3,data4], ignore_index=True)
data = data.rename(columns={"Proven victims":"proven_victims","Possible victims":"possible_victims" })

# Dicccionario para ISO A3 code para países
country_dict = pd.read_csv('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv',
                           encoding = 'UTF-8')

## Limpieza de país

Genero pais 1, 2 y 3 para campos con más de un país. Se eliminan los que tienen más de 3 países para simplicidad del análisis

In [112]:
# Generar pais 1-2-3
pais = data["Country"].str.split("\r\n", n = 2, expand = True)

data['pais_1'] = pais[0]
data['pais_2'] = pais[1]
data['pais_3'] = pais[2]

cols = ['pais_1','pais_2','pais_3']

for col in cols:
    data[col] = (data[col].str.replace('United States of America','United States ').replace(' \(suspected\)',"", regex=True)
    .replace('Soviet Union','Russia').replace("United States Mexico","United States").replace(' \(claimed\)',"", regex=True).replace(" \(alleged\)'',", regex=True).replace("East Germany","Germany")
    .replace("West Germany","Germany", regex=True).replace('Austria-Hungary',"Austria").replace("^East Germany","Germany",  regex=True).replace('German Empire','Germany')
    .replace('Allied-occupied Germany',"Germany").replace('Kingdom of Romania',"Romania")
    )


# Limpieza específica de pais 2 y 3
data.pais_2 = (data.pais_2.replace(' \(alleged\)',"", regex=True).replace("\r\n.*","", regex=True).replace('\n.*',"",regex=True)
)
data.pais_3 = (data.pais_3.replace(' \(alleged\)',"", regex=True).replace("\r\n.*","", regex=True).replace('\n.*',"",regex=True)
)


## Limpieza campo año

Limpio años y divido en año de inicio y año de termino.

In [113]:
# limpio datos 
data['Years active'] = (data['Years active'].replace('late ',"", regex=True).replace("s","", regex=True).replace(' and earlier',"", regex=True)
.replace("\?","", regex=True).replace('c.',"", regex=True).replace('30 June ',"", regex=True).replace('preent',"2020", regex=True)
.replace('23 July ',"", regex=True)
)

# divido en año inicio y termino

años  = data["Years active"].str.split(" to ", expand = True)
data['año_inicio'] = pd.to_numeric(años[0],"ignore")
data['año_termino'] = pd.to_numeric(años[1], "ignore")

In [148]:
for i in range(0,len(data.año_termino)):
    if pd.isna(data.año_termino[i]):
        data.año_termino[i] = data.año_inicio[i]

data['año_termino'] = data['año_termino'].map(int)

## Limpieza campo "víctimas probadas" y "posibles víctimas"

Limpieza del campo `proven_victims` y `possible_victims` . Algunos campos tienen rangos, "+", "~", etc. 

* Víctimas probadas

In [135]:
data.proven_victims = (data.proven_victims.replace(".*–","", regex=True).replace("\\+","", regex=True)
)
data.proven_victims = pd.to_numeric(data.proven_victims)

* Vícitimas posibles

In [143]:
data.possible_victims = (data.possible_victims.replace(".*–","", regex=True).replace("\\+","", regex=True)
.replace("Unknown","0").replace("\\?","", regex = True).replace("-","", regex = True)
.replace("\\~","", regex=True)
)
data.possible_victims = pd.to_numeric(data.possible_victims)

# reemplazo valores nulos o 0
for i in range(0,len(data.possible_victims)):
    if pd.isna(data.possible_victims[i]) or data.possible_victims[i]== 0:
        data.possible_victims[i] = data.proven_victims[i]

data['possible_victims'] = data['possible_victims'].map(int)

In [151]:
data.head()

,Name,Country,Years active,proven_victims,possible_victims,Notes,pais_1,pais_2,pais_3,año_inicio,año_termino
0,Karl Denke,Germany,1900 to 1924,30,42,Killed and cannibalized poor travelers and hom...,Germany,None,None,1900,1924
1,Francisco das Chagas Rodrigues de Brito,Brazil,1989 to 2003,30,42,"Pedophile who sexually abused, murdered and mu...",Brazil,None,None,1989,2003
2,Luis Gregorio Ramírez Maestre,Colombia,2010 to 2013,30,30,Killed motorists in various municipalities bef...,Colombia,None,None,2010,2013
3,David Thabo Simelane,Swaziland,2000 to 2001,28,45,Sexually assaulted women he befriended in fore...,Swaziland,None,None,2000,2001
4,Zhang Jun,China,1993 to 2000,28,28,"Robbed 22 stores in several Chinese provinces,...",China,None,None,1993,2000
